In [ ]:
import sys
!{sys.executable} -m pip install openai


In [ ]:
import openai
import os
import json

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
def chat(input):
    completion = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        temperature = 0.5,
        messages = [
            {"role": "user", "content": input},
        ]
    )
    return completion.choices[0].message.content

def instruct(input, temperature = 0.5, stop = None):
    completion = openai.Completion.create(
        model = "gpt-3.5-turbo-instruct",
        temperature = temperature,
        prompt = input,
        max_tokens = 1000,
        stop = stop
    )
    return completion.choices[0].text 

In [ ]:
def fuzzy_if(input, condition):
    prompt = f"""You are an advanced conditional evaluator. Your job is to take an string and a conditional statement as input, and return a bool if the input string satisfies the conditional. The inputs may be in forms of words, sentences, numbers, or even paragraphs, and the conditionals may be querying the content or the form of the input. The conditionals may also be in the form of statements ("is a complete sentence", for example) or questions ("is it about environmentalism?", for example), and you must return true or false in any of these scenarios.
    Let's try it out. Remember to only return true or false.
    Input: {input}
    Conditional: {condition}
    Output:"""
    output = instruct(prompt, temperature = 0, stop = "\n").strip().lower()
    if output == "true":
        return True
    elif output == "false":
        return False
    else:
        raise ValueError("Unexpected output: " + output)
    
def fuzzy_filter(input, condition):
    prompt = f"""You are an advanced semantic computer. Your job is to take a string and a semantic find condition as input, and return a $-separated list of substrings that satisfy the condition. The inputs may be in the form of sentences, lists, paragraphs, or even entire documents, and the find conditions may be related to form or content of the input, such as asking for all characters in a story, all dates in a sentence, all descriptions of a person, or all sentences that are questions. Remeber to return a $-separated list of substrings that satisfy the condition.
    Let's try it out.
    Input: {input}
    Conditional: {condition}
    Output:"""
    output = instruct(prompt, temperature = 0).strip().split("$")
    return output

def fuzzy_foreach(input, condition, action):
    split_input = fuzzy_filter(input, condition)
    results = []
    for element in split_input:
        prompt = f"Perform the following action on the input. Follow the action directly and do not include any extra commentary.\nInput: {element}\nAction: {action}\nOutput:"
        result = instruct(prompt, temperature = 0).strip()
        results.append(result)
    return results
    


In [ ]:
test_suite = {
    "dog": [["is fuzzy", True],["is a mammal", True], ["is a reptile", False], ["is a dog", True], ["is a cat", False], ["is a pet", True], ["is a plant", False], ["is a living thing", True], ["is a non-living thing", False], ["is a human", False], ["is a dog or a cat", True], ["is a dog and a cat", False], ["references time in some way",False],["has oppsoable thumbs",False],["can talk",False]],
    "four score and seven years ago": [["is this a beginning of a famous quote",True],["from a famous book",False], ["is this a complete sentence?",False],["is colorful",False],["is a number",False],["is a date",True],["is a year or a date or a number",True], ["references time in some way",True],["about a character",False],["has oppsoable thumbs",False],["can talk",False]],
    "the quick brown fox jumps over the lazy dog": [["is this a beginning of a famous quote",False], ["is this a complete sentence?",True],["is colorful",False],["is a number",False],["is a date",False],["is a year or a date or a number",False], ["references time in some way",False],["about a character",True],["has oppsoable thumbs",False],["can talk",False]],
    "the king was pregnant" : [["is this a beginning of a famous quote",False], ["from a famous book",True], ["is this a complete sentence?",True],["is colorful",False],["is a number",False],["is a date",False],["is a year or a date or a number",False], ["references time in some way",False],["about a character",True],["has oppsoable thumbs",False],["can talk",False]],
    "long black flowy dress, with a slit up the side": [["is this a beginning of a famous quote",False], ["from a famous book",False], ["is this a complete sentence?",False],["is colorful",True],["is a number",False],["is a date",False],["is a year or a date or a number",False], ["references time in some way",False],["about a character",False],["has oppsoable thumbs",False],["can talk",False]]
}

In [ ]:
def test_fuzzy_if():
    pass_count = 0
    fail_count = 0

    for input, conditions in test_suite.items():
        for condition, expected_output in conditions:
            result = fuzzy_if(input, condition)
            if result == expected_output:
                pass_count += 1
                print(f"✅ PASS: Input: {input}, Condition: {condition}, Output: {result}")
            else:
                fail_count += 1
                print(f"😡 FAIL: Input: {input}, Condition: {condition}, Expected: {expected_output}, Got: {result}")

    print(f"\nTotal tests: {pass_count + fail_count}, Pass %: {pass_count/(pass_count+fail_count)} (Passed: {pass_count}, Failed: {fail_count})")

test_fuzzy_if()

In [ ]:
# fuzzy_if(input, testing_condition)
print(fuzzy_if("a man a plan a canal panama", "was once president"))

In [ ]:
print(fuzzy_if("recent polling showing biden down in michigan", "is good news for america"))

In [ ]:
print(fuzzy_if("louie armstrong", "went to the moon"))
print(fuzzy_if("neil armstrong", "went to the moon"))
print(fuzzy_if("lance armstrong", "went to the moon"))

In [ ]:
print(fuzzy_if("a man a plan a canal panama", "is a palindrome"))

In [ ]:
# fuzzy_filter(input, filter_criteria)
print(fuzzy_filter("a man a plan a canal panama", "nouns"))

In [ ]:
fuzzy_filter(someHackerNewsComment, "bad opinions")[0] == someHackerNewsComment
#resolves to true for all values of someHackerNewsComment

In [ ]:
print(fuzzy_filter("a man a plan a canal panama", "groups of 2 words"))

In [ ]:
# fuzzy_foreach(input, split_condition, action)
print(fuzzy_foreach("a man a plan a canal panama", "nouns", "write it backwards"))

In [ ]:
# fuzzy_foreach(somgLongListOfUserFeedback, "things that relate to feature N", "give the piece of user feedback a priority of 1, 2, or 3 given our criteria of X, Y, and Z, then repeat back the feedback")
